## Step 14: 같은 변수 반복 사용

현재 DeZero에는 문제점이 하나 있다. 바로 같은 변수를 반복 사용했을 때 정상적으로 작동하지 못한다는 점인데, `y = add(x, x)`와 같은 계산을 할 경우 미분을 제대로 하지 못한다.

```python
x = Variable(np.array(3.0))
y = add(x, x)
print(y.data)

y.backward()
print(x.grad)
```

\> 6.0

\> 1.0

x를 3.0으로 설정했기 때문에 x.grad는 2가 되어야 한다. 하지만 실행 결과 x.grad가 1.0이라고 출력된다.

### 14.1 문제의 원인

이러한 문제는 왜 발생했을까? 이는 Variable 클래스에서 일어난다.

```python
class Variable:
    ...

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx # 문제 발생 구간!

                if x.creator is not None:
                    funcs.append(x.creator)
```

현재 구현에서는 출력 쪽에서 전해지는 미분값을 그대로 대입한다. 따라서 같은 변수를 사용하면 전파되는 미분값을 덮어쓰고 있는 것이다.

### 14.2 해결책

해당 부분에 예외 처리를 두어 `x.grad = None`일 때는 지금처럼, 아닐 때는 전달된 미분값을 더하도록 구현하면 된다. 

```python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    funcs.append(x.creator)
```

여기서 왜 `x.grad += gx` 대신 `x.grad = x.grad + gx`를 사용했는지 의문이 들 수 있다. 이는 ndarray 인스턴스의 복사(copy)와 덮어쓰기(overwrite)의 차이에 의한 것으로, 인플레이스 연산(in-place operation)인 += 연산을 사용하면 문제가 발생하기 때문이다.

덮어쓰기 연산을 사용할 경우, x.grad와 y.grad가 같은 id의 객체 값을 참조하게 되어, y.grad에 잘못된 값이 저장되게 된다. 따라서 정상적인 계산을 수행하고자 하는 경우 복사 연산을 사용해서 서로 다른 값을 참조하도록 해야 한다.

이와 같이 Variable 클래스를 수정할 경우 같은 변수를 반복해서 사용할 수 있다.

```python
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)
```

\> 2.0

x를 세 번 사용해도 정상적으로 계산이 진행된다.

```python
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)
```

\> 3.0

### 14.3 미분값 재설정

다만, 여기서도 아직 해결해야 할 문제가 있다. x를 사용하여 연산을 수행한 후, 이어서 다른 계산에서도 같은 변수를 사용하면 할당된 미분값이 초기화되지 않은 채로 계산이 시작되어 잘못된 결과가 나온다. 따라서 이 부분을 초기화하는 로직을 추가해보겠다.

```python
class Variable:
    ...
    def cleargrad(self):
        self.grad = None
```

cleargrad 메서드는 이름 그대로 미분값을 초기화하는 메서드로, 호출 시 self.grad에 None을 대입하게 된다.

```python
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 미분값 초기화
x.cleargrad()

y = add(add(x, x), x)
y.backward()
print(x.grad)
```

\> 2.0

\> 3.0